# Mounting Azure Data Lake Gen2 Storage(ADLS) using Service Principal

## Steps:
1. Get client-id, tenant-id, client-key from secret scope
2. Set Spark Config
3. Call File System Utility Mount to mount Storage

In [0]:
def mount_adls_storage(storage_name, container_name):
    client_id = dbutils.secrets.get(scope="formulaone-scope",key="client-id")
    client_key = dbutils.secrets.get(scope="formulaone-scope",key="client-secret")
    tenant_id = dbutils.secrets.get(scope="formulaone-scope",key="tenant-id")

    configs = {"fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": client_id,
        "fs.azure.account.oauth2.client.secret": client_key,
        "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    if any(mount.mountPoint == f"/mnt/{storage_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_name}/{container_name}")

    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_name}/{container_name}",
        extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls_storage("formula1datalake280199", "raw")

In [0]:
mount_adls_storage("formula1datalake280199", "processed")

In [0]:
mount_adls_storage("formula1datalake280199", "presentation")